- Homework2
- **This homework focuses on training and evaluating prediction models for a particular problem and dataset.**
- **In this analysis, we focus on using the data collected by CDC to build a data analytics solution for death risk prediction.**
- The goal in this homework is to work with the data to build and evaluate prediction models that capture the relationship between the descriptive features and the target feature **death_yn**

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 

In [4]:
df = pd.read_csv(r"C:\Users\35385\Desktop\CS_Spring2024\DataAnal\Homework2\new_features_df1.csv", keep_default_na=True, delimiter=",", skipinitialspace=True, encoding="Windows-1252")

In [6]:
df.head(10)

,case_month,res_state,age_group,sex,race,ethnicity,hosp_yn,death_yn,county_frequency_size,party,Profile,seasons
0,2020-04-01,CA,65+ years,Male,White,Non-Hispanic/Latino,Yes,Yes,Medium,DEMOCRAT,65+ years_White_Male_Non-Hispanic/Latino,Spring
1,2020-03-01,NY,65+ years,Female,White,Non-Hispanic/Latino,Yes,Yes,Medium,DEMOCRAT,65+ years_White_Female_Non-Hispanic/Latino,Spring
2,2021-09-01,FL,65+ years,Female,White,Non-Hispanic/Latino,No,Yes,Medium,DEMOCRAT,65+ years_White_Female_Non-Hispanic/Latino,Fall
3,2020-07-01,TX,50 to 64 years,Male,White,Hispanic/Latino,Yes,Yes,Very_Large,DEMOCRAT,50 to 64 years_White_Male_Hispanic/Latino,Summer
4,2020-12-01,AZ,65+ years,Male,White,Hispanic/Latino,Yes,Yes,Small,DEMOCRAT,65+ years_White_Male_Hispanic/Latino,Winter
5,2020-11-01,WI,65+ years,Female,White,Non-Hispanic/Latino,Yes,Yes,Medium,REPUBLICAN,65+ years_White_Female_Non-Hispanic/Latino,Fall
6,2020-05-01,MD,65+ years,Female,White,Non-Hispanic/Latino,No,Yes,Small,DEMOCRAT,65+ years_White_Female_Non-Hispanic/Latino,Spring
7,2020-11-01,MN,65+ years,Male,White,Non-Hispanic/Latino,No,Yes,Small,DEMOCRAT,65+ years_White_Male_Non-Hispanic/Latino,Fall
8,2021-02-01,NY,65+ years,Male,White,Non-Hispanic/Latino,No,Yes,Huge,DEMOCRAT,65+ years_White_Male_Non-Hispanic/Latino,Winter
9,2020-05-01,NY,65+ years,Female,White,Non-Hispanic/Latino,No,Yes,Huge,DEMOCRAT,65+ years_White_Female_Non-Hispanic/Latino,Spring


In [8]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame containing your dataset
x = df.drop(columns=['death_yn'])  # Features
y = df['death_yn']  # Target variable

# Split the dataset into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# x_train and y_train are your training data and labels, respectively
# x_test and y_test are your test data and labels, respectively

In [9]:
print("\nDescriptive features in X:\n", x)
print("\nTarget feature in y:\n", y)


Descriptive features in X:
        case_month res_state       age_group     sex   race  \
0      2020-04-01        CA       65+ years    Male  White   
1      2020-03-01        NY       65+ years  Female  White   
2      2021-09-01        FL       65+ years  Female  White   
3      2020-07-01        TX  50 to 64 years    Male  White   
4      2020-12-01        AZ       65+ years    Male  White   
...           ...       ...             ...     ...    ...   
39942  2020-04-01        PA  18 to 49 years    Male  White   
39943  2021-07-01        SC    0 - 17 years  Female  White   
39944  2020-11-01        ID  18 to 49 years  Female  White   
39945  2022-05-01        MN    0 - 17 years  Female  White   
39946  2021-09-01        NV    0 - 17 years    Male  White   

                 ethnicity hosp_yn county_frequency_size       party  \
0      Non-Hispanic/Latino     Yes                Medium    DEMOCRAT   
1      Non-Hispanic/Latino     Yes                Medium    DEMOCRAT   
2      Non

In [13]:
features = ['party']
linreg = LinearRegression().fit(x[features], y)

ValueError: could not convert string to float: 'DEMOCRAT'

: 